# Model

In [21]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Import merged data: 

In [33]:
X_train=pd.read_csv("X_train.csv")
y_train=pd.read_csv("y_train.csv")
X_test=pd.read_csv("X_test.csv")
y_test=pd.read_csv("y_test.csv")

##  Data Processing:

In [34]:
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb

In [35]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_df2 = X_train2.select_dtypes(include=numerics)
num_col2=num_df2.columns
num_col2

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_df = X_train.select_dtypes(include=numerics)
num_col=num_df.columns


In [36]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
col_2 = X_train2.select_dtypes(exclude=numerics)
col2=col_2.columns


In [37]:
keep2=["NAME_CONTRACT_TYPE","CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY"]
keep5=["NAME_FAMILY_STATUS"]
rest=[]
for i in cat_col:
    if (i not in keep2 ) and (i not in keep5):
        rest+=[i]


In [38]:
pre_col = ColumnTransformer(
    remainder="passthrough",
    transformers=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'),num_col),
                  ('ohe_2',OneHotEncoder(max_categories=2,handle_unknown='ignore', dtype='int'),keep2),
                  ("ohe_5",OneHotEncoder(max_categories=5,handle_unknown='ignore', dtype='int'),keep5),
                  ("ohe_rest",OneHotEncoder(handle_unknown='ignore', dtype='int'),rest)
                 ])

pre_col_2 = ColumnTransformer(
    remainder="passthrough",
    transformers=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'),num_col2),
                  ("ohe_rest",OneHotEncoder(handle_unknown='ignore', dtype='int'),col2)
                 ])

## Data2

In [39]:
X_train2=pd.read_csv("X_train_new.csv")
X_test2=pd.read_csv("X_test_new.csv")

## Manual merge EXPORT

In [429]:
grace_train=pd.DataFrame()
grace_pred=pd.DataFrame()

## LightGBM: 

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
import lightgbm as lgb

In [ ]:
# gird search results:

#{'lgb__class_weight': {0: 1, 1: 9},
 #'lgb__learning_rate': 0.06,
 #'lgb__max_depth': 17,
 #'lgb__n_estimators': 150,
 #'lgb__num_leaves': 25}

#{'lgb__class_weight': {0: 0.08, 1: 0.92},
 #'lgb__learning_rate': 0.06,
 #'lgb__max_depth': 14,
 #'lgb__n_estimators': 200,
 #'lgb__num_leaves': 15}

#{'lgb__class_weight': {0: 0.08, 1: 0.92},
# 'lgb__learning_rate': 0.06,
# 'lgb__max_depth': 14,
# 'lgb__n_estimators': 200,
# 'lgb__num_leaves': 15}

#{'lgb__class_weight': {0: 0.08, 1: 0.92},
# 'lgb__learning_rate': 0.06,
# 'lgb__max_depth': 10,
# 'lgb__n_estimators': 200,
# 'lgb__num_leaves': 15}

In [41]:
pl_lgbm2=Pipeline([
    ("pre_col2", pre_col_2),
    ('lgb', lgb.LGBMClassifier(max_depth=10,
                               learning_rate=0.06,
                               n_estimators=200,
                               num_leaves=15,
                               class_weight={0:0.08,1:0.92}))
])


pl_lgbm2.fit(X_train2.replace([np.inf, -np.inf], np.nan), y_train)

Pipeline(steps=[('pre_col2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer', SimpleImputer(),
                                                  Index(['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'NAME_CONTRACT_STATUS_Completed_prev', 'NAME_CONTRACT_S...
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object'))])),
                ('lgb',
                 LGBMClassifier(class_weight={0: 0.08, 1: 0.92},
                                learning_rate=0.06, max_depth=10,
                                n_estimators=200, num_leaves=15))])

In [43]:
from sklearn.model_selection import cross_val_score
cross_val_score(pl_lgbm2,X_train2.replace([np.inf, -np.inf], np.nan), y_train, cv=5,scoring="roc_auc")

array([0.75981792, 0.76182963, 0.76784037, 0.76150257, 0.77037915])

In [45]:
pl_lgbm2.predict(X_test2.replace([np.inf, -np.inf], np.nan))

array([1, 0, 0, ..., 0, 0, 0])

In [47]:
# light gbm

In [46]:
roc_auc_score(y_test, pl_lgbm2.predict(X_test2))

0.6954753053307428

In [318]:
from sklearn.model_selection import GridSearchCV
grid_lol = GridSearchCV(pl_lgbm2, grid_lgbm, cv=5,verbose=10,scoring="roc_auc")
#grid_lol.fit(X_train2.replace([np.inf, -np.inf], np.nan), y_train)

### Featuer selection:

In [89]:
pre_col.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)
estimator = lgb.LGBMClassifier(class_weight={0:0.08,1:0.92})
selector = RFE(estimator, n_features_to_select=350,verbose=10)
#selector.fit(pre_col.transform(X_train.replace([np.inf, -np.inf], np.nan)), y_train)

In [101]:
#selected=selector.transform(pre_col.transform(X_train.replace([np.inf, -np.inf], np.nan)))

In [102]:
estimator=lgb.LGBMClassifier(max_depth=10,
                               learning_rate=0.08,
                               n_estimators=200,
                               num_leaves=15,)
#class_weight={0:0.08,1:0.92}
#estimator.fit(selected,y_train)

In [103]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(estimator,selected, y_train, cv=5,scoring="roc_auc")

## Nueral Network: 

In [48]:
from sklearn.neural_network import MLPClassifier

In [49]:
pl_nn=Pipeline([
    ("pre_col", pre_col),
    ('nn', MLPClassifier())])

#nn_weight = [1 if i==0 else 10 for i in y_train['TARGET'].to_numpy()]
#pl_nn.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train, sample_weight=nn_weight)

In [114]:
#nn_weight = [1 if i==0 else 10 for i in y_train['TARGET'].to_numpy()]
#a=pre_col.transform(X_train.replace([np.inf, -np.inf], np.nan))
#nn=MLPClassifier()
#nn.fit(a,y_train, sample_weight=nn_weight)

In [50]:
#pl_nn.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)

In [ ]:
# bad validation set performance

## Ada Boost:

In [118]:
from sklearn.ensemble import AdaBoostClassifier

In [120]:
e1=lgb.LGBMClassifier(max_depth=10,class_weight={0:0.08,1:0.92})


pl_ada=Pipeline([
    ("pre_col", pre_col),
    ('ada', AdaBoostClassifier(estimator=e1,))
])

In [121]:
#pl_ada.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)

In [ ]:
# bad validation set performance

## Guassian Mixture

In [136]:
from sklearn.mixture import GaussianMixture

In [137]:
pl_gm=Pipeline([
    ("pre_col", pre_col),
    ('gm',GaussianMixture(n_components=2,reg_covar=0.04,weights_init=[0.1,0.9]))])

#nn_weight = [1 if i==0 else 10 for i in y_train['TARGET'].to_numpy()]
#pl_nn.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train, sample_weight=nn_weight)

In [138]:
#pl_gm.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)

In [139]:
# bad validation set performance

## HistGradientBoostingClassifier:

In [82]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
#{'hb__class_weight': {0: 0.08, 1: 0.92},
 #'hb__l2_regularization': 10,
# 'hb__learning_rate': 0.08,
# 'hb__loss': 'binary_crossentropy',
 #'hb__max_depth': 8,
# 'hb__max_iter': 150,
 #'hb__scoring': 'roc_auc'}
#class_weight={0:1,1:11.5}

In [83]:
pl_hgb=Pipeline([
    ("pre_col", pre_col),
    ('hb', HistGradientBoostingClassifier(loss="binary_crossentropy",
                                          scoring="roc_auc",
                                          max_iter=150,
                                          max_depth=5,
                                          l2_regularization=10,
                                          class_weight={0:1,1:11.5}))
])

pl_hgb2=Pipeline([
    ("pre_col", pre_col_2),
    ('hb', HistGradientBoostingClassifier(loss="binary_crossentropy",
                                          scoring="roc_auc",
                                          max_iter=150,
                                          learning_rate=0.08,
                                          max_depth=8,
                                          l2_regularization=10,
                                          class_weight={0:1,1:11.5}))
])


In [52]:
#pl_hgb.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train,hb__sample_weight=nn_weight)
#pl_hgb2.fit(X_train2.replace([np.inf, -np.inf], np.nan), y_train,)

In [439]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(pl_hgb, X_train.replace([np.inf, -np.inf], np.nan), y_train, cv=5,scoring="roc_auc",error_score="raise")

In [ ]:
# pretty good validation performance

In [ ]:
# final performance:

In [441]:
roc_auc_score(y_test, pl_hgb2.predict(X_test2))

0.6956520951455987

## ExtraTreesClassifier: 0.6627

In [57]:
from sklearn.ensemble import ExtraTreesClassifier

In [58]:
pre_col2 = ColumnTransformer(
    remainder="passthrough",
    transformers=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'),num_col),
                  ('ohe_2',OneHotEncoder(handle_unknown='ignore', dtype='int'),keep2),
                  ("ohe_5",OneHotEncoder(handle_unknown='ignore', dtype='int'),keep5),
                  ("ohe_rest",OneHotEncoder(handle_unknown='ignore', dtype='int'),rest)
                 ])

pl_et=Pipeline([
    ("pre_col", pre_col),
    ('hb', ExtraTreesClassifier(max_features="sqrt",
                                class_weight="balanced_subsample",
                                max_depth=7,
                                min_samples_leaf=2,
                                criterion="log_loss",
                                n_estimators=300
                               ))
])

pl_et2=Pipeline([
    ("pre_col", pre_col_2),
    ('hb', ExtraTreesClassifier(max_features="sqrt",
                                class_weight="balanced",
                                max_depth=7,
                                min_samples_leaf=2,
                                criterion="log_loss",
                                n_estimators=300
                               ))
])


In [59]:
#pl_et2.fit(X_train2.replace([np.inf, -np.inf], np.nan), y_train)

Pipeline(steps=[('pre_col',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer', SimpleImputer(),
                                                  Index(['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'NAME_CONTRACT_STATUS_Completed_prev', 'NAME_CONTRACT_ST...
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object'))])),
                ('hb',
                 ExtraTreesClassifier(class_weight='balanced',
                                      criterion='log_loss', max_depth=7,
                                      min_samples_leaf=2, n_estimators=300))])

In [447]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(pl_hgb, X_train.replace([np.inf, -np.inf], np.nan), y_train, cv=5,scoring="roc_auc",error_score="raise")

In [54]:
# final performce:

In [60]:
roc_auc_score(y_test, pl_et2.predict(X_test2))

0.6720468435583442

## Some feature selection:Randeom Forrest

In [61]:
pre_col.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)
es = RandomForestClassifier(max_depth = 6,
                                  n_estimators=250,
                                  ccp_alpha=0,
                                  random_state=42,
                                  class_weight="balanced")

selector = RFE(es, n_features_to_select=375,verbose=10)

In [62]:
#selector.fit(pre_col.transform(X_train.replace([np.inf, -np.inf], np.nan)), y_train)

In [63]:
#selector.estimator_

In [64]:
#pre_col.fit(X_train.replace([np.inf, -np.inf], np.nan))
#selected=selector.transform(pre_col.transform(X_train.replace([np.inf, -np.inf], np.nan)))

In [65]:
from sklearn.preprocessing import FunctionTransformer

In [70]:
def select_features(X, bool_list):
    return X[:, bool_list]

#transformer = FunctionTransformer(select_features, kw_args={'bool_list': selector.estimator_})

#pl_fs = Pipeline([
    #("pre_col", pre_col),
    #('feature_selection', transformer),
    #('rf_feature_selection', RandomForestClassifier(max_depth = 6,
                                  #n_estimators=250,
                                  #ccp_alpha=0,
                                  #class_weight={0:0.08,1:0.92}))
#])

#pl_fs.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)

In [71]:
#pl_fs.predict(X_test.replace([np.inf, -np.inf], np.nan)).sum()

In [72]:
#roc_auc_score(y_test, pl_fs.predict(X_test))

## STACKING:

In [73]:
from sklearn.ensemble import StackingClassifier

In [77]:
#estimators = [
    #('lgbm', pl),
    #('hgb',pl_hgb),
    #("rf_fs",pl_rf),
    #('et',pl_et),
#]

# ('et',pl_et),
#fe=lgb.LGBMClassifier(max_depth=10,
                               #learning_rate=0.08,
                               #n_estimators=200,
                               #num_leaves=15,
                               #class_weight={0:0.08,1:0.92})

#fe_lr=LogisticRegression(class_weight={0:0.8,1:11.5})

#reg=StackingClassifier(estimators,final_estimator=fe)

In [78]:
#pl.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)
#pl_hgb.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)
#reg.fit(X_train.replace([np.inf, -np.inf], np.nan), y_train)

In [203]:
#pl.predict(X_test.replace([np.inf, -np.inf], np.nan)).sum()

## Final Stack: 

In [86]:
from sklearn.ensemble import StackingClassifier
estimators = [
    ('lgbm', pl_lgbm2),
    ('hgb',pl_hgb2)
]

fe=lgb.LGBMClassifier(max_depth=10,
                               learning_rate=0.08,
                               n_estimators=150,
                               num_leaves=15,
                               class_weight={0:0.08,1:0.92})

reg2=StackingClassifier(estimators,final_estimator=fe)

In [85]:
reg2.fit(X_train2.replace([np.inf, -np.inf], np.nan), y_train)

StackingClassifier(estimators=[('lgbm',
                                Pipeline(steps=[('pre_col2',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('imputer',
                                                                                  SimpleImputer(),
                                                                                  Index(['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'NAME_CONTRACT_...
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object'))])),
                                                ('hb',
                                                 HistGradientBoostingClassifier(class_weight={0: 1,
                                                                                              1: 11.5},
                                                                                l2_regularization=10,
                                                                                learning_rate=0.08,
                                                                                loss='binary_crossentropy',
                                                                                max_depth=8,
                                                                                max_iter=150,
                                                                                scoring='roc_auc'))]))],
                   final_estimator=LGBMClassifier(class_weight={0: 0.08,
                                                                1: 0.92},
                                                  learning_rate=0.08,
                                                  max_depth=10,
                                                  n_estimators=150,
                                                  num_leaves=15))

In [87]:
#reg2.predict(X_test2.replace([np.inf, -np.inf], np.nan)).sum()

In [89]:
#roc_auc_score(y_test, reg2.predict(X_test2))
# best 0.702, mostly 0.701+